In [49]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout, Dense, Flatten, Reshape, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
import numpy as np

In [50]:
from keras.utils import load_img, img_to_array, 

In [18]:
!mkdir gen_bobross_images

In [51]:
images_path = "my_img"

In [113]:
import os
images= []
for image in os.listdir(images_path):
    images.append(img_to_array(load_img(images_path +'/'+ image,target_size=(64,64))))
    

In [114]:
img_width= 64
img_height= 64
channels= 3
img_shape= (img_width, img_height, channels)
latent_dim= 100
adam= Adam(learning_rate= 0.0002)

In [115]:
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 8 * 8, input_dim= latent_dim))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Reshape((8, 8, 256)))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2D(3, (3,3), activation= 'tanh', padding= 'same'))
    
    model.summary()
    return model

generator= build_generator()

Model: "sequential_6"
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16384)             1654784   
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 16384)             0         
                                                                 
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16384)             1654784   
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 16384)             0         
                                                                 
 reshape_2 (Reshape)         (None, 8, 8, 256)         0         
 reshape_2 (Reshape)         (None, 8, 8, 256)         0         
                                

In [116]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same', input_shape= img_shape))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation= 'sigmoid'))    
    
    model. summary()
    return model

discriminator= build_discriminator()
discriminator.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

Model: "sequential_7"
Model: "sequential_7"
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 64, 64, 64)        1792      
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 64)        1792      
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_12 (Conv2D)          (None, 64, 64, 128)       73856     
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_12 (Conv2D)          (No

In [117]:

gan = Sequential()
gan.add(generator)
discriminator.trainable= False
gan.add(discriminator)

gan.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

In [120]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name= 0.00000000

def save_image(epoch):
    r, c = 5, 5 
    noise= np.random.normal(0,1, (r*c, latent_dim))
    gen_imgs= generator.predict(noise)
    global save_name
    save_name+= 0.00000001
    print("%.8f" % save_name)
    gen_imgs= 0.5 * gen_imgs + 0.5
    fig, axs= plt.subplots(r,c)
    cnt= 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0])
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("gen_bobross_images/%.8f.png" % save_name)
    plt.close()


In [ ]:
def train(epochs, batch_size= 64, save_interval= 200):
    x= np.array(images)
    x= x/ 127.5 -1.
    valid= np.ones((batch_size, 1))
    fake= np.zeros((batch_size, 1))
    batch_per_epoch= int(x.shape[0] /batch_size)
    
    for epoch in range(epochs):
        for j in range(batch_per_epoch):
            idx= np.random.randint(0, x.shape[0], batch_size)(x,_),(_,_)= cifar10.load_data()
            imgs= x[idx]

            noise= np.random.normal(0,1, (batch_size, latent_dim))
            gen_imgs= generator.predict(noise)

            # train discriminator
            d_loss_real= discriminator.train_on_batch(imgs, valid)
            d_loss_fake= discriminator.train_on_batch(gen_imgs, fake)
            d_loss= np.add(d_loss_real,d_loss_fake) * 0.5

            noise= np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss= gan.train_on_batch(noise, valid)

            print(f'{epoch} [D loss {d_loss} ] [G loss {g_loss}]')
        
        save_image(epoch)
        
train(30000,8)

1/1 [==============================] - 0s 262ms/step
0 [D loss [6.72884789e-06 1.00000000e+00] ] [G loss [49.38306427001953, 0.0]]
0 [D loss [6.72884789e-06 1.00000000e+00] ] [G loss [49.38306427001953, 0.0]]
1/1 [==============================] - 0s 301ms/step
0 [D loss [0.55113184 0.9375    ] ] [G loss [42.72710418701172, 0.0]]
0 [D loss [0.55113184 0.9375    ] ] [G loss [42.72710418701172, 0.0]]
1/1 [==============================] - 0s 334ms/step
0 [D loss [0.70339674 0.9375    ] ] [G loss [59.178382873535156, 0.0]]
0 [D loss [0.70339674 0.9375    ] ] [G loss [59.178382873535156, 0.0]]
1/1 [==============================] - 0s 320ms/step
0 [D loss [7.28614759e-05 1.00000000e+00] ] [G loss [48.884517669677734, 0.0]]
0 [D loss [7.28614759e-05 1.00000000e+00] ] [G loss [48.884517669677734, 0.0]]
1/1 [==============================] - 0s 275ms/step
0 [D loss [2.48835884e-19 1.00000000e+00] ] [G loss [43.370792388916016, 0.0]]
0 [D loss [2.48835884e-19 1.00000000e+00] ] [G loss [43.3707

1 [D loss [131.63470984   0.5       ] ] [G loss [2.4043850898742676, 0.875]]
1 [D loss [131.63470984   0.5       ] ] [G loss [2.4043850898742676, 0.875]]
1/1 [==============================] - 0s 323ms/step
1 [D loss [1.71105567e-05 1.00000000e+00] ] [G loss [30.157649993896484, 0.0]]
1 [D loss [1.71105567e-05 1.00000000e+00] ] [G loss [30.157649993896484, 0.0]]
1/1 [==============================] - 0s 340ms/step
1 [D loss [2.91034416 0.8125    ] ] [G loss [5.190339088439941, 0.125]]
1 [D loss [2.91034416 0.8125    ] ] [G loss [5.190339088439941, 0.125]]
1/1 [==============================] - 0s 415ms/step
1 [D loss [36.23377609  0.0625    ] ] [G loss [1.0599110478182183e-09, 1.0]]
1 [D loss [36.23377609  0.0625    ] ] [G loss [1.0599110478182183e-09, 1.0]]
1/1 [==============================] - 0s 495ms/step
1 [D loss [0.03357963 1.        ] ] [G loss [34.229026794433594, 0.0]]
1 [D loss [0.03357963 1.        ] ] [G loss [34.229026794433594, 0.0]]
1/1 [==============================]

1/1 [==============================] - 0s 285ms/step
2 [D loss [10.93521047  0.625     ] ] [G loss [67.93364715576172, 0.125]]
2 [D loss [10.93521047  0.625     ] ] [G loss [67.93364715576172, 0.125]]
1/1 [==============================] - 0s 299ms/step
2 [D loss [22.40028  0.125  ] ] [G loss [55.98538589477539, 0.25]]
2 [D loss [22.40028  0.125  ] ] [G loss [55.98538589477539, 0.25]]
1/1 [==============================] - 0s 290ms/step
2 [D loss [3.99759965 0.875     ] ] [G loss [212.86019897460938, 0.125]]
2 [D loss [3.99759965 0.875     ] ] [G loss [212.86019897460938, 0.125]]
1/1 [==============================] - 0s 300ms/step
2 [D loss [31.4146347  0.75     ] ] [G loss [178.4122314453125, 0.25]]
2 [D loss [31.4146347  0.75     ] ] [G loss [178.4122314453125, 0.25]]
1/1 [==============================] - 0s 301ms/step
2 [D loss [17.82194686  0.75      ] ] [G loss [134.9884490966797, 0.125]]
2 [D loss [17.82194686  0.75      ] ] [G loss [134.9884490966797, 0.125]]
1/1 [============

3 [D loss [16.51475525  0.875     ] ] [G loss [113.29299926757812, 0.0]]
3 [D loss [16.51475525  0.875     ] ] [G loss [113.29299926757812, 0.0]]
1/1 [==============================] - 0s 310ms/step
3 [D loss [2.16946808e-14 1.00000000e+00] ] [G loss [139.05160522460938, 0.0]]
3 [D loss [2.16946808e-14 1.00000000e+00] ] [G loss [139.05160522460938, 0.0]]
1/1 [==============================] - 0s 298ms/step
3 [D loss [9.68296149e-07 1.00000000e+00] ] [G loss [159.82615661621094, 0.0]]
3 [D loss [9.68296149e-07 1.00000000e+00] ] [G loss [159.82615661621094, 0.0]]
1/1 [==============================] - 0s 310ms/step
3 [D loss [0.38617316 0.9375    ] ] [G loss [137.5860595703125, 0.0]]
3 [D loss [0.38617316 0.9375    ] ] [G loss [137.5860595703125, 0.0]]
1/1 [==============================] - 0s 309ms/step
3 [D loss [0.0043201 1.       ] ] [G loss [161.48501586914062, 0.0]]
3 [D loss [0.0043201 1.       ] ] [G loss [161.48501586914062, 0.0]]
1/1 [==============================] - 0s 303ms/

4 [D loss [2.20095984 0.875     ] ] [G loss [22.234359741210938, 0.0]]
1/1 [==============================] - 0s 287ms/step
4 [D loss [0.23669886 0.9375    ] ] [G loss [38.01007843017578, 0.0]]
4 [D loss [0.23669886 0.9375    ] ] [G loss [38.01007843017578, 0.0]]
1/1 [==============================] - 0s 297ms/step
4 [D loss [0.84003927 0.875     ] ] [G loss [56.053611755371094, 0.0]]
4 [D loss [0.84003927 0.875     ] ] [G loss [56.053611755371094, 0.0]]
1/1 [==============================] - 0s 307ms/step
4 [D loss [3.29363489 0.6875    ] ] [G loss [48.459083557128906, 0.0]]
4 [D loss [3.29363489 0.6875    ] ] [G loss [48.459083557128906, 0.0]]
1/1 [==============================] - 0s 305ms/step
4 [D loss [2.38462354 0.8125    ] ] [G loss [39.620018005371094, 0.0]]
4 [D loss [2.38462354 0.8125    ] ] [G loss [39.620018005371094, 0.0]]
1/1 [==============================] - 0s 294ms/step
4 [D loss [2.19517884 0.75      ] ] [G loss [29.00360107421875, 0.125]]
4 [D loss [2.19517884 0.75

...

1/1 [==============================] - 0s 289ms/step
49 [D loss [0. 1.] ] [G loss [278263.625, 0.0]]
49 [D loss [0. 1.] ] [G loss [278263.625, 0.0]]
1/1 [==============================] - 0s 288ms/step
49 [D loss [0. 1.] ] [G loss [233252.078125, 0.0]]
49 [D loss [0. 1.] ] [G loss [233252.078125, 0.0]]
1/1 [==============================] - 0s 303ms/step
49 [D loss [388.3531189   0.9375   ] ] [G loss [45809.4765625, 0.0]]
49 [D loss [388.3531189   0.9375   ] ] [G loss [45809.4765625, 0.0]]
1/1 [==============================] - 0s 302ms/step
49 [D loss [0. 1.] ] [G loss [22861.046875, 0.0]]
49 [D loss [0. 1.] ] [G loss [22861.046875, 0.0]]
1/1 [==============================] - 0s 304ms/step
49 [D loss [464.33181763   0.8125    ] ] [G loss [206318.25, 0.0]]
49 [D loss [464.33181763   0.8125    ] ] [G loss [206318.25, 0.0]]
1/1 [==============================] - 0s 281ms/step
49 [D loss [0. 1.] ] [G loss [806335.0, 0.0]]
49 [D loss [0. 1.] ] [G loss [806335.0, 0.0]]
1/1 [===============

1/1 [==============================] - 0s 296ms/step
51 [D loss [0. 1.] ] [G loss [1569152.5, 0.0]]
51 [D loss [0. 1.] ] [G loss [1569152.5, 0.0]]
1/1 [==============================] - 0s 299ms/step
51 [D loss [0. 1.] ] [G loss [1312665.375, 0.0]]
51 [D loss [0. 1.] ] [G loss [1312665.375, 0.0]]
1/1 [==============================] - 0s 315ms/step
51 [D loss [0. 1.] ] [G loss [1138736.25, 0.0]]
51 [D loss [0. 1.] ] [G loss [1138736.25, 0.0]]
1/1 [==============================] - 0s 305ms/step
51 [D loss [0. 1.] ] [G loss [1017915.875, 0.0]]
51 [D loss [0. 1.] ] [G loss [1017915.875, 0.0]]
1/1 [==============================] - 0s 297ms/step
51 [D loss [0. 1.] ] [G loss [935678.375, 0.0]]
51 [D loss [0. 1.] ] [G loss [935678.375, 0.0]]
1/1 [==============================] - 0s 295ms/step
51 [D loss [0. 1.] ] [G loss [876836.875, 0.0]]
51 [D loss [0. 1.] ] [G loss [876836.875, 0.0]]
1/1 [==============================] - 0s 316ms/step
51 [D loss [0. 1.] ] [G loss [830478.0, 0.0]]
51 [D

1/1 [==============================] - 0s 298ms/step
52 [D loss [0. 1.] ] [G loss [716428.9375, 0.0]]
52 [D loss [0. 1.] ] [G loss [716428.9375, 0.0]]
1/1 [==============================] - 0s 314ms/step
52 [D loss [0. 1.] ] [G loss [715148.5625, 0.0]]
52 [D loss [0. 1.] ] [G loss [715148.5625, 0.0]]
1/1 [==============================] - 0s 300ms/step
52 [D loss [0. 1.] ] [G loss [718290.375, 0.0]]
52 [D loss [0. 1.] ] [G loss [718290.375, 0.0]]
1/1 [==============================] - 0s 281ms/step
52 [D loss [0. 1.] ] [G loss [713300.3125, 0.0]]
52 [D loss [0. 1.] ] [G loss [713300.3125, 0.0]]
1/1 [==============================] - 0s 274ms/step
52 [D loss [0. 1.] ] [G loss [715880.8125, 0.0]]
52 [D loss [0. 1.] ] [G loss [715880.8125, 0.0]]
1/1 [==============================] - 0s 303ms/step
52 [D loss [0. 1.] ] [G loss [716539.375, 0.0]]
52 [D loss [0. 1.] ] [G loss [716539.375, 0.0]]
1/1 [==============================] - 0s 303ms/step
52 [D loss [0. 1.] ] [G loss [718848.375, 0.0

54 [D loss [0. 1.] ] [G loss [712795.125, 0.0]]
54 [D loss [0. 1.] ] [G loss [712795.125, 0.0]]
1/1 [==============================] - 0s 302ms/step
54 [D loss [0. 1.] ] [G loss [711499.5, 0.0]]
54 [D loss [0. 1.] ] [G loss [711499.5, 0.0]]
1/1 [==============================] - 0s 288ms/step
54 [D loss [0. 1.] ] [G loss [716190.0625, 0.0]]
54 [D loss [0. 1.] ] [G loss [716190.0625, 0.0]]
1/1 [==============================] - 0s 282ms/step
54 [D loss [0. 1.] ] [G loss [717200.375, 0.0]]
54 [D loss [0. 1.] ] [G loss [717200.375, 0.0]]
1/1 [==============================] - 0s 311ms/step
54 [D loss [0. 1.] ] [G loss [717957.5625, 0.0]]
54 [D loss [0. 1.] ] [G loss [717957.5625, 0.0]]
1/1 [==============================] - 0s 324ms/step
54 [D loss [0. 1.] ] [G loss [718970.4375, 0.0]]
54 [D loss [0. 1.] ] [G loss [718970.4375, 0.0]]
1/1 [==============================] - 0s 324ms/step
54 [D loss [0. 1.] ] [G loss [716449.8125, 0.0]]
1/1 [==============================] - 0s 475ms/step
54

KeyboardInterrupt: 

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('gen_bobross_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
noise = np.random.normal(0, 1, (16, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs[2])

In [ ]:
generator.save_weights("/gen_bobross_images/generator1hour.h5")
discriminator.save_weights("/gen_bobross_images/discriminator1hour.h5")